In [1]:
import numpy as np
import psutil
import resource
from dask.distributed import Client, as_completed

In [2]:
n_workers = 5
client = Client(n_workers=n_workers)
futures = as_completed(None, with_results=True)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 5
Total threads: 15,Total memory: 30.98 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42355,Workers: 5
Dashboard: http://127.0.0.1:8787/status,Total threads: 15
Started: Just now,Total memory: 30.98 GiB
Comm: tcp://127.0.0.1:36925,Total threads: 3
Dashboard: http://127.0.0.1:44191/status,Memory: 6.20 GiB
Nanny: tcp://127.0.0.1:39185,


In [3]:
BIG_NUM = 99999999999999999
BIG_NUMS = [BIG_NUM]*1000

In [4]:
def run_fake_md():
    return BIG_NUMS

In [9]:
for i in range(10):
    j = client.submit(run_fake_md, pure=False)
    futures.add(j)

it = 0
while it < 1000000:  
    md_items = next(futures)[1]
    # if it%100 == 0:
    #     client.restart()
    j = client.submit(run_fake_md, pure=False)
    futures.add(j)
    print(it, psutil.Process().memory_info().rss/10**9, resource.getrusage(resource.RUSAGE_SELF).ru_maxrss, psutil.virtual_memory().percent, flush=True)
    it += 1
    # print(it)

while len(futures.futures):
    md_items = next(futures)[1]
    # print(it)
    print(it, psutil.Process().memory_info().rss/10**9, resource.getrusage(resource.RUSAGE_SELF).ru_maxrss, psutil.virtual_memory().percent, flush=True)
    it += 1


0 0.253743104 253200 46.0
1 0.253743104 253200 46.0
2 0.253743104 253200 46.0
3 0.253743104 253200 46.0
4 0.253743104 253200 46.0


2023-02-24 14:50:28,214 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2023-02-24 14:50:28,713 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2023-02-24 14:50:30,855 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2023-02-24 14:50:34,213 - distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
2023-02-24 14:50:35,183 - distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)


KeyboardInterrupt: 

In [11]:
client.close()